In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc
rc('text', usetex=True)
%matplotlib inline

In [ ]:
def fourier_second_derivative(u, dx):

    uhat = np.fft.fft(u)
    nyquist_freq = np.pi/dx
    dk = nyquist_freq/(nx/2)
    k = np.arange(float(nx))
    k[:nx/2] = k[:nx/2]*dk
    k[nx/2:] = k[:nx/2]-nyquist_freq
    duhat = (1j*k)**2*uhat
    ddu = np.real(np.fft.ifft(duhat))
    
    
    return ddu

In [ ]:
#Specify spatial grid

xmin = -10
xmax = 10
nx = 256
x = np.linspace(xmin, xmax, nx)
dx = x[1] - x[0]

#Specify temporal grid and stability constant
v = 1
stability_constant = 0.2
dt = stability_constant*dx/v
nt = 2400
t = np.arange(0, nt)*dt

#Initialize empty array for displacements (u(x, t_{i})) to be dumped. Specify boundary conditions

u_initial = np.zeros(len(x))
u = np.zeros((len(u_initial), nt))
u_initial[0] = 0
u_initial[-1] = 0
u[:, 0] = u_initial
uold = u_initial


#Define source term (time dependent amplitude with a Gaussian Pulse in the spatial domain)

A = 500*np.sin(8*(t-4))*np.exp(-(t-4)**2)
sigma = 1.5*dx
source = np.exp(-x**2/sigma**2)

In [ ]:
#Time marching scheme

for t in range(nt-1):

    ddu = fourier_second_derivative(u[:, t], dx)
    unew = v**2*dt**2*ddu + 2*u[:, t] - uold
    unew = unew + A[t]*source*dt**2
    unew[0] = 0
    unew[-1] = 0

    u[:, t+1] = unew
    uold = u[:, t]




In [ ]:
#Initialize axis for plotting (this will be updated during the animation portion)

fig = plt.figure()
ax = plt.axes(xlim=(-12, 12), ylim=(-5, 5))
ax.set_xlabel(r'Position ($x$)')
ax.set_ylabel(r'Displacement ($u(x, t)$)')
plt.title(r'1D Wave Equation with Source: Fixed Boundaries')
line, = ax.plot([], [], lw=2, alpha = 0.8)

In [ ]:
#Define animation functions

def init():
    line.set_data([], [])
    return line,

def animate(i):
    
    xmin = -10
    xmax = 10
    nx = 256
    x = np.linspace(xmin, xmax, nx)
    y = u[:, i]
    line.set_data(x, y)
    return line,

In [ ]:
#Animate and save as mp4

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames= nt, interval=1, blit=True)

anim.save('1D_acoustic_wave_source_term.mp4', fps=30, extra_args=['-vcodec', 'libx264'])